In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from html_to_markdown import convert_to_markdown
from IPython.display import display, Markdown

In [ ]:
from scifeeder.utils import read_papers_csv
from scifeeder.types import Paper
papers=list(read_papers_csv('suba_j.csv'))

In [ ]:
from collections import Counter
counts = Counter()
for p in papers:
    if p.issn and p.doi:
        counts[p.issn]+=1

In [ ]:
from scifeeder.selenium_cls import Selenium
from scifeeder.issn import DATA

In [ ]:
paper = None
for idx, p1 in enumerate(reversed(papers)):
    if p1.issn:
        if p1.issn in {"0253-9772"}:
            continue
    if (p1.issn not in DATA) and p1.doi and p1.year > 1980 and counts[p1.issn] > 0:
        paper = p1
        break
print(idx, len(papers), len(counts), len(DATA))
if paper:
    paper.issn in DATA

In [ ]:
paper = papers[204]

In [ ]:
f"https://doi.org/{paper.doi}"

In [ ]:
paper

In [ ]:
def toline(p):
    return f'"{p.issn}": {p.key},'
print(toline(paper))

In [ ]:
from scifeeder.selenium_cls import Selenium, Soup, UndetectedSelenium

from scifeeder.issn import DATA

assert paper.issn in DATA and paper.doi, paper
# web = UndetectedSelenium(timeout=15, headless=True)
web = Selenium(timeout=15, headless=False)
markdown, soup = web.run(paper.doi, DATA[paper.issn])
if markdown:
    with open('saved.md', 'wt') as fp:
        fp.write(markdown)
Markdown(markdown)

In [ ]:
assert paper.issn in DATA and paper.doi, paper
markdown, soup = web.run(paper.doi, DATA[paper.issn])
if markdown:
    with open('saved.md', 'wt') as fp:
        fp.write(markdown)
Markdown(markdown)

In [ ]:
markdown is None

In [ ]:
HTML(web.find_html())

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(web.find_html(),"lxml")
print(soup.prettify())

In [ ]:
c=web.pdf(DATA[p.issn])
if c is not None:
    with open('saved.pdf', 'wb') as fp:
        fp.write(c)

In [ ]:
soup=web.resoupify()
pages = soup.select('[id="viewer"] .page')

Markdown(web.tofrag(web.find_html(), Location('[id="viewer"] .page')))

----

## EPMC

In [ ]:
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from html_to_markdown import convert_to_markdown
from IPython.display import HTML, Markdown

In [ ]:
from scifeeder.epmc import EPMC
from scifeeder.xml_utils import Events, PMCEvents, PMCTAGS

In [ ]:
from IPython.display import HTML

In [ ]:
DONE= {'PMC5486124', 'PMC5447075', 'PMC5853234', 'PMC5490910', 'PMC5490927', 'PMC5823009', 'PMC5399074',
      'PMC5575511', 'PMC5387099', 'PMC5405513', 'PMC5412409', 'PMC5462045', 'PMC5444443', 'PMC5435427'}

In [ ]:

for p in reversed(papers):
    if p.pmcid and p.pmcid not in DONE and p.year > 2010:
        break
p

In [ ]:
f"https://doi.org/{p.doi}"

In [ ]:
CSS="""
<style>
.email { color:orange; }
.title-group { color: blue; font-size: 14pt; }
figure { border: solid orange 1px; padding: 0.5em; }
.missing { font-size:larger; color: red; }
.sc { color: red; font-weight: bold; }
</style>
"""

In [ ]:
x =  EPMC.from_pmcid(p.pmcid)
html = x.html()
if x.missing:
    print(x.missing)
HTML(CSS + html)

In [ ]:
# x.save_content('saved.xml')

In [ ]:
x.save_content("saved.xml", pretty=True)